# Exploratory Data Analysis

#### We load in our data, reshape and merge

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np

In [ ]:
basics = pd.read_csv(r"C:/Users/jungn/OneDrive/Documents/movie project/title.basics.tsv.gz", sep = "\t", compression = "gzip", na_values="\\N")
ratings = pd.read_csv(r"C:/Users/jungn/OneDrive/Documents/movie project/title.ratings.tsv.gz", sep = "\t", compression = "gzip")

C:\Users\jungn\AppData\Local\Temp\ipykernel_23336\1574588733.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  basics = pd.read_csv(r"C:/Users/jungn/OneDrive/Documents/movie project/title.basics.tsv.gz", sep = "\t", compression = "gzip", na_values="\\N")


#### from the IMDB dataset we only want movies (between 1980-2022), so we exclude tv shows, videos etc

In [ ]:
movies = basics[(basics["titleType"] == "movie") & (basics["startYear"].between(1980, 2022))]
[["tconst", "primaryTitle", "startYear", "runtimeMinutes", "genres"]]

[['tconst', 'primaryTitle', 'startYear', 'runtimeMinutes', 'genres']]

#### We now merge the movies data and ratings data based on "tconst" which is the unique movie ID

In [ ]:
movies = movies.merge(ratings, on="tconst", how="left")

#### Load in the principals data 

In [ ]:
principals = pd.read_csv(r"C:/Users/jungn/OneDrive/Documents/movie project/title.principals.tsv.gz", sep="\t", na_values="\\N")

#### We get the director count and cast size

In [ ]:
principals = principals[principals["category"].isin(["director", "actor", "actress"])]

In [ ]:
director_counts = (principals[principals["category"] == "director"]
    .groupby("tconst")
    .size()
    .rename("num_directors")
    .reset_index()
)

In [ ]:
cast_size = (principals[principals["category"].isin(["actor", "actress"])]
    .groupby("tconst")
    .size()
    .rename("cast_size")
    .reset_index()
)

In [ ]:
movies = (movies
    .merge(director_counts, on="tconst", how="left")
    .merge(cast_size, on="tconst", how="left")
)


In [ ]:
movies[["num_directors", "cast_size"]] = (movies[["num_directors", "cast_size"]].fillna(0))


#### Convert into a csv file 

In [ ]:
movies.to_csv("C:/Users/jungn/OneDrive/Documents/movie project/movies_features.csv", index=False)


In [ ]:
films = pd.read_csv("C:/Users/jungn/OneDrive/Documents/movie project/movies_features.csv")

In [ ]:
films

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,num_directors,cast_size
0,tt0011801,movie,Tötet nicht mehr,Tötet nicht mehr,0,2019.0,NaN,NaN,"Action,Crime",NaN,NaN,1.0,10.0
1,tt0015724,movie,Dama de noche,Dama de noche,0,1993.0,NaN,102.0,"Drama,Mystery,Romance",6.2,35.0,1.0,10.0
2,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118.0,"Comedy,Fantasy,Romance",6.4,92914.0,1.0,10.0
3,tt0036606,movie,"Another Time, Another Place","Another Time, Another Place",0,1983.0,NaN,118.0,"Drama,War",6.4,379.0,1.0,10.0
4,tt0038086,movie,Shiva und die Galgenblume,Shiva und die Galgenblume,0,1993.0,NaN,NaN,Thriller,7.0,30.0,1.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
395550,tt9916622,movie,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,0,2015.0,NaN,57.0,Documentary,NaN,NaN,1.0,1.0
395551,tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,De la ilusión al desconcierto: cine colombiano...,0,2007.0,NaN,100.0,Documentary,NaN,NaN,1.0,0.0
395552,tt9916706,movie,Dankyavar Danka,Dankyavar Danka,0,2013.0,NaN,NaN,Comedy,7.7,9.0,1.0,10.0
395553,tt9916730,movie,6 Gunn,6 Gunn,0,2017.0,NaN,116.0,Drama,7.0,13.0,1.0,8.0


#### Now we move on to some exploring

In [ ]:
# Total NaN count:
films.isna().any(axis=1).sum()


np.int64(395555)

In [ ]:
# Which columns have NaNs and how many?
films.isna().sum().sort_values(ascending=False)


endYear           395555
averageRating     168946
numVotes          168946
runtimeMinutes     89259
genres             30256
primaryTitle           2
originalTitle          2
tconst                 0
titleType              0
isAdult                0
startYear              0
num_directors          0
cast_size              0
dtype: int64

Above, we see that numVotes and averageRating have the same number of NaNs which means they are missing together. Movies without an IMDB rating is an indication that they are very obscure, and since this project is about how a movie ages in audience pereption these will not serve any purpose so they can be dropped.

In [ ]:
films = films.dropna(subset = ["averageRating", "numVotes"])
films = films.dropna(subset = ["primaryTitle"])
# drop the 2 primaryTitle NaNs as it is a negligible amount

I have decided to keep the rest (genres, runtimeMinutes) as they are still important and will prevent potential bias towards mainstream cinema.

#### Handling runtime NaNs:

In [ ]:
films["runtime_missing"] = films["runtimeMinutes"].isna().astype(int)
films["runtimeMinutes"] = films["runtimeMinutes"].fillna(films["runtimeMinutes"].median())
# Create a new column indicating which rows were NaNs as an integer (0,1) 
# Replace missing values with median runtime of the dataset (mean is not as robust when it comes to extreme values or outliers)

#### Handling genre NaNs:

In [ ]:
films["genres"] = films["genres"].fillna("Unknown") # Replace NaN with Unknown
films["has_genre_info"] = (films["genres"] != "Unknown").astype(int) # Column: Genre info exists, 1. Genre info does not exist, 0.

In [ ]:
films = films.drop("endYear", axis=1)

In [ ]:
films.isna().sum()

tconst             0
titleType          0
primaryTitle       0
originalTitle      0
isAdult            0
startYear          0
runtimeMinutes     0
genres             0
averageRating      0
numVotes           0
num_directors      0
cast_size          0
runtime_missing    0
has_genre_info     0
dtype: int64

We've dropped a lot of rows, but we still have more than 220 thousand rows to work with which is ample data for this project.

In [ ]:
films

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes,num_directors,cast_size,runtime_missing,has_genre_info
1,tt0015724,movie,Dama de noche,Dama de noche,0,1993.0,102.0,"Drama,Mystery,Romance",6.2,35.0,1.0,10.0,0,1
2,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,118.0,"Comedy,Fantasy,Romance",6.4,92914.0,1.0,10.0,0,1
3,tt0036606,movie,"Another Time, Another Place","Another Time, Another Place",0,1983.0,118.0,"Drama,War",6.4,379.0,1.0,10.0,0,1
4,tt0038086,movie,Shiva und die Galgenblume,Shiva und die Galgenblume,0,1993.0,92.0,Thriller,7.0,30.0,1.0,10.0,1,1
5,tt0038687,movie,Let There Be Light,Let There Be Light,0,1980.0,58.0,"Documentary,War",7.5,2147.0,0.0,1.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395547,tt9916362,movie,Coven,Akelarre,0,2020.0,92.0,"Drama,History,Horror",6.4,6167.0,1.0,10.0,0,1
395548,tt9916428,movie,The Secret of China,Hong xing zhao yao Zhong guo,0,2019.0,92.0,"Adventure,History,War",4.7,23.0,1.0,10.0,1,1
395549,tt9916538,movie,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,0,2019.0,123.0,Drama,7.6,12.0,1.0,10.0,0,1
395552,tt9916706,movie,Dankyavar Danka,Dankyavar Danka,0,2013.0,92.0,Comedy,7.7,9.0,1.0,10.0,1,1


#### Before moving on guns blazing, it's important to think about the overall idea:
- What make a movie a "classic"?
- Given the data:
    - High ratings
    - High review counts
    - runtimes
- These are a good indication of "well known" or "famous"

We will explore how many movies in the dataset have an average rating of 7.5 or higher:

In [ ]:
films[films.averageRating >= 7.5].shape[0]

41555

Let's see how many of these movies have review counts in the thousands:

In [ ]:
films.loc[(films["averageRating"] >= 7.5) & (films["numVotes"] >= 1000)].shape[0]

3846

Not many movies are left relatively speaking.